In [79]:
import subprocess
import os
import re
from pprint import pprint

In [80]:
js_getVARS_loc = os.path.join(os.path.abspath('parsergen'), "get_vars.js")
php_getVARS_loc = os.path.join(os.path.abspath('parsergen'), "get_vars.php")
php_samples_loc = os.path.join(os.path.abspath('parsergen'), "php_samples")
js_samples_loc = os.path.join(os.path.abspath('parsergen'), "js_samples")

In [81]:
print(js_getVARS_loc)
print(php_getVARS_loc)
print(php_samples_loc)
print(js_samples_loc)

/home/srn/Desktop/bloom-experiment-lab/dataprocessing/parsergen/get_vars.js
/home/srn/Desktop/bloom-experiment-lab/dataprocessing/parsergen/get_vars.php
/home/srn/Desktop/bloom-experiment-lab/dataprocessing/parsergen/php_samples
/home/srn/Desktop/bloom-experiment-lab/dataprocessing/parsergen/js_samples


In [82]:
# Get all files with parent directory
php_samples = [os.path.join(php_samples_loc, f) for f in os.listdir(php_samples_loc) if os.path.isfile(os.path.join(php_samples_loc, f))]
js_samples = [os.path.join(js_samples_loc, f) for f in os.listdir(js_samples_loc) if os.path.isfile(os.path.join(js_samples_loc, f))]

print(js_samples)
print(php_samples)

['/home/srn/Desktop/bloom-experiment-lab/dataprocessing/parsergen/js_samples/sample1.js']
['/home/srn/Desktop/bloom-experiment-lab/dataprocessing/parsergen/php_samples/sample1.php']


In [83]:
js_get_vars_sourcetype = "module"
node_cmd = ["node", js_getVARS_loc, js_get_vars_sourcetype, js_samples[0]]
php_cmd = ["php", php_getVARS_loc, php_samples[0]]

js_vars = []
php_vars = []

try:
    node_output = subprocess.check_output(node_cmd, stderr=subprocess.PIPE).decode('utf-8')
    js_vars = node_output.split(',')
    php_output = subprocess.check_output(php_cmd, stderr=subprocess.PIPE).decode('utf-8')
    php_vars = php_output.split(',')
except subprocess.CalledProcessError as e:
    print(e.stderr)


In [84]:
print(php_vars)

["$array['']", "$array['1']", '$array', "$_GET['userData']", '$tainted', '$sanitized', '$query', '$ret']


In [85]:
def code_cleaner(filename):
    with open(filename, 'r') as f:
        code = f.read()

    # GENERAL: 
    # remove multiline comments
    code = re.sub(r'/\*(.*?)\*/', '', code, flags=re.DOTALL)
    # remove all single line comments (//|#) except if (//|#) is inside of a string like "htes // asdf" or 'htes # asdf'
    code = re.sub(r'(?<!\\)(["\'])(?:\\.|(?!\1).)*?\1|//.*?$|#.*?$', 
                  lambda m: m.group(0) if m.group(0).startswith('"') or m.group(0).startswith("'") else '', code, flags=re.MULTILINE)
    # remove all newlines after a ( ,|.|\(|\[ ) or spaces after a ( ,|.|\(|\[ )
    code = re.sub(r'(\[|\(|,|\.)\s+', r'\1', code)
    # remove all newlines before a ( ,|.|;|\)|\] ) or spaces before a ( ,|.|;|\)|\] )
    code = re.sub(r'\s+(\]|\)|,|\.|;)', r'\1', code)
    # remove all trailing comma before a ( \) | \] )
    code = re.sub(r',(\s*[\]\)])', r'\1', code)
    # split code into lines
    code = code.split('\n')
    # remove leading and trailing whitespace
    code = [line.strip() for line in code]
    # remove all semi-colons at the end of a line
    code = [re.sub(r'(;|{)$', '', line).strip() for line in code]
    
    # After removing aliens, we can remove some twigs symbols, single words and numbers
    # remove all elements that are one word or numeric only in a string or symbols only in a string
    code = [line for line in code if not re.match(r'^\W+$', line) 
                                        and not re.match(r'^\w+$', line) 
                                            and not re.match(r'^\d+$', line)]

    # PHP:
    # remove all php tags
    code = [line for line in code if not line.startswith('<?php') and not line.startswith('?>')]

    # lastly remove all empty lines
    code = list(filter(None, code))

    return code


In [86]:
js_code = code_cleaner(js_samples[0])
js_code

["const express = require('express')",
 "const crypto = require('crypto')",
 'const app = express()',
 "const db = require('better-sqlite3')('db.sqlite3')",
 'db.exec(`DROP TABLE IF EXISTS users;`)',
 'db.exec(`CREATE TABLE users(id INTEGER PRIMARY KEY,username TEXT,password TEXT);`)',
 'const FLAG = process.env.FLAG || "dice{test_flag}"',
 'const PORT = process.env.PORT || 3000',
 'const users = [...Array(100_000)].map(() => ({ user: `user-${crypto.randomUUID()}`,pass: crypto.randomBytes(8).toString("hex") }))',
 'db.exec(`INSERT INTO users (id,username,password) VALUES ${users.map((u,i) => `(${i},\'${u.user}\',\'${u.pass}\')`).join(",")}`)',
 'const isAdmin = {}',
 'const newAdmin = users[Math.floor(Math.random() * users.length)]',
 'isAdmin[newAdmin.user] = true',
 'app.use(express.urlencoded({ extended: false }))',
 'app.use(express.static("public"))',
 'app.post("/api/login",(req,res) =>',
 'const { user,pass } = req.body',
 'console.log("[REQUEST BODY] : ",req.body)',
 "const que

In [87]:
php_code = code_cleaner(php_samples[0])
php_code

['$array = array()',
 "$array[] = 'safe'",
 "$array[] = $_GET['userData']",
 "$array[] = 'safe'",
 '$tainted = $array[1]',
 '$sanitized = filter_var($tainted,FILTER_SANITIZE_EMAIL)',
 'if (filter_var($sanitized,FILTER_VALIDATE_EMAIL))',
 '$tainted = $sanitized',
 '$tainted = ""',
 '$query = "cat \'".$tainted."\'"',
 '$ret = system($query)']

In [88]:
# Extract the variable references of JS and PHP
def extract_vars_references(vars, code):
    references = []
    for var in vars:
        # case sensitive and match whole word only or if wrapped in a special character
        var_pattern = r'(?<!\w)' + re.escape(var) + r'(?!\w)'
        
        var_references = []
        for line in code:
            if re.search(var_pattern, line):
                var_references.append(line)
        references.append((var, var_references))

    return references

In [89]:
js_vars_references = extract_vars_references(js_vars, js_code)
print(js_vars)
pprint(js_vars_references)

['express', 'crypto', 'app', 'db', 'FLAG', 'PORT', 'users', 'isAdmin', 'newAdmin', 'user', 'pass', 'query', 'id']
[('express',
  ["const express = require('express')",
   'const app = express()',
   'app.use(express.urlencoded({ extended: false }))',
   'app.use(express.static("public"))']),
 ('crypto',
  ["const crypto = require('crypto')",
   'const users = [...Array(100_000)].map(() => ({ user: '
   '`user-${crypto.randomUUID()}`,pass: crypto.randomBytes(8).toString("hex") '
   '}))']),
 ('app',
  ['const app = express()',
   'app.use(express.urlencoded({ extended: false }))',
   'app.use(express.static("public"))',
   'app.post("/api/login",(req,res) =>',
   'app.listen(PORT,() => console.log(`web/funnylogin listening on port '
   '${PORT}`))']),
 ('db',
  ["const db = require('better-sqlite3')('db.sqlite3')",
   'db.exec(`DROP TABLE IF EXISTS users;`)',
   'db.exec(`CREATE TABLE users(id INTEGER PRIMARY KEY,username TEXT,password '
   'TEXT);`)',
   'db.exec(`INSERT INTO users (id

In [90]:
php_vars_references = extract_vars_references(php_vars, php_code)
print(php_vars)
pprint(php_vars_references)

["$array['']", "$array['1']", '$array', "$_GET['userData']", '$tainted', '$sanitized', '$query', '$ret']
[("$array['']", []),
 ("$array['1']", []),
 ('$array',
  ['$array = array()',
   "$array[] = 'safe'",
   "$array[] = $_GET['userData']",
   "$array[] = 'safe'",
   '$tainted = $array[1]']),
 ("$_GET['userData']", ["$array[] = $_GET['userData']"]),
 ('$tainted',
  ['$tainted = $array[1]',
   '$sanitized = filter_var($tainted,FILTER_SANITIZE_EMAIL)',
   '$tainted = $sanitized',
   '$tainted = ""',
   '$query = "cat \'".$tainted."\'"']),
 ('$sanitized',
  ['$sanitized = filter_var($tainted,FILTER_SANITIZE_EMAIL)',
   'if (filter_var($sanitized,FILTER_VALIDATE_EMAIL))',
   '$tainted = $sanitized']),
 ('$query', ['$query = "cat \'".$tainted."\'"', '$ret = system($query)']),
 ('$ret', ['$ret = system($query)'])]


In [91]:
def check_variable_usage(code_snippet):
    # php regex rules for catching tainted variables with user input
    php_pattern = re.compile(r'\b(?:php|http)://|(?:(\$_(?:GET|POST|REQUEST|SERVER|COOKIE|ENV|FILES)\b)|\b(?:GET|POST|REQUEST|SERVER|COOKIE|ENV|FILES)\b)\b')
    # pure js regex rules for catching tainted variables with user input
    js_pattern = re.compile(r'(?:req|request)\.(?:body|params|query|headers)', re.IGNORECASE)
    # express js regex rules for catching tainted variables with user input
    express_js_pattern = re.compile(r'(?:req|request)\.(?:body|params|query|headers|param|queryparam|get|post|paramfrom)', re.IGNORECASE)

    php_match = re.search(php_pattern, code_snippet)
    js_match = re.search(js_pattern, code_snippet)
    express_js_match = re.search(express_js_pattern, code_snippet)

    return bool(php_match), bool(js_match), bool(express_js_match)


In [92]:

def get_tainted_variables(references):
    tainted_variables = set()

    for var, snippets in references:
        for snippet in snippets:
            matches = check_variable_usage(snippet)
            if any(matches):
                tainted_variables.add(var)

    return list(tainted_variables)


In [93]:
tainted_js_vars = get_tainted_variables(js_vars_references)
tainted_php_vars = get_tainted_variables(php_vars_references)

print("Tainted JS")
pprint(tainted_js_vars)
print()
print("Tainted PHP")
pprint(tainted_php_vars)


Tainted JS
['user', 'pass']

Tainted PHP
['$array', "$_GET['userData']"]


In [94]:
def extract_tainted_snippets(references, tainted_variables, variables):
    regex_pattern = r'(?<![a-zA-Z0-9_]){}(?![a-zA-Z0-9_])'

    # Function to update the tainted variables list
    def update_tainted_variables(new_tainted_vars):
        for var in new_tainted_vars:
            if var not in tainted_variables:
                tainted_variables.append(var)

    # Iterate over each variable in the variables list
    for var, snippets in references:
        for snippet in snippets:
            for variable in variables:
                # Check if the variable is used in the snippet
                if variable in snippet and variable != var:
                    # Add the variable to the references if it's not already present
                    if not any(variable == v[0] for v in references):
                        references.append((variable, []))
                    # Add the variable to the tainted variables list if it's not already present
                    if variable not in tainted_variables:
                        tainted_variables.append(variable)

    # Extract tainted snippets for each tainted variable
    tainted_var_and_snippets = []
    for tainted_var in tainted_variables:
        tainted_snippets = []
        for var, snippets in references:
            if var == tainted_var:
                tainted_snippets.extend(snippets)
        tainted_var_and_snippets.append((tainted_var, tainted_snippets))

    return tainted_var_and_snippets

In [105]:
tainted_js_var_and_code = extract_tainted_snippets(js_vars_references, tainted_js_vars, js_vars)
tainted_php_var_and_code = extract_tainted_snippets(php_vars_references, tainted_php_vars, php_vars)


pprint(tainted_js_var_and_code)
print()
pprint(tainted_php_var_and_code)

[('user',
  ['const users = [...Array(100_000)].map(() => ({ user: '
   '`user-${crypto.randomUUID()}`,pass: crypto.randomBytes(8).toString("hex") '
   '}))',
   'db.exec(`INSERT INTO users (id,username,password) VALUES ${users.map((u,i) '
   '=> `(${i},\'${u.user}\',\'${u.pass}\')`).join(",")}`)',
   'isAdmin[newAdmin.user] = true',
   'const { user,pass } = req.body',
   "const query = `SELECT id FROM users WHERE username = '${user}' AND "
   "password = '${pass}';`",
   'if (users[id] && isAdmin[user])']),
 ('pass',
  ['const users = [...Array(100_000)].map(() => ({ user: '
   '`user-${crypto.randomUUID()}`,pass: crypto.randomBytes(8).toString("hex") '
   '}))',
   'db.exec(`INSERT INTO users (id,username,password) VALUES ${users.map((u,i) '
   '=> `(${i},\'${u.user}\',\'${u.pass}\')`).join(",")}`)',
   'const { user,pass } = req.body',
   "const query = `SELECT id FROM users WHERE username = '${user}' AND "
   "password = '${pass}';`"]),
 ('app',
  ['const app = express()',
   'app

In [96]:
def sqli_vulnerability(data):
    sql_syntax_pattern = re.compile(r'\b(?:SELECT|INSERT|UPDATE|DELETE|FROM|WHERE|AND|OR)\b', re.IGNORECASE)
    concatenated_string_pattern = re.compile(r'["\']\s*\.\s*["\']')

    # Initialize counters for SQL injection patterns
    sql_syntax_count = 0
    concatenated_string_count = 0

    # Combine snippets from all categories into a single list
    all_snippets = [snippet for _, snippets in data for snippet in snippets]

    # Iterate over each snippet and detect SQLi vulnerabilities
    for snippet in all_snippets:
        if re.search(sql_syntax_pattern, snippet):
            sql_syntax_count += 1
        if re.search(concatenated_string_pattern, snippet):
            concatenated_string_count += 1

    if sql_syntax_count > 0:
        sql_syntax_count = 1
    if concatenated_string_count > 0:
        concatenated_string_count = 1

    # Return a single list containing the count of patterns found
    return [sql_syntax_count, concatenated_string_count]

In [97]:
def xss_vulnerability(data):
    html_tag_pattern = re.compile(r'<(?:'
        # Opening HTML tags for common XSS-vulnerable elements
        r'script|img|iframe|frame|a|form|input|textarea|svg'
        # Additional attributes for some tags
        r'(?:\s+[a-z0-9_-]+(?:\s*=\s*(?:"[^"]*"|\'[^\']*\'|[^>\s]+))?)*\s*/?'
        # Closing HTML tags for script, iframe, frame, a, form, textarea
        r'|\/(?:script|iframe|frame|a|form|textarea)'
        r')>'
    )
    concatenated_string_pattern = re.compile(r'["\']\s*\.\s*["\']')

    html_tag_count = 0
    concatenated_string_count = 0

    # Combine snippets from all categories into a single list
    all_snippets = [snippet for _, snippets in data for snippet in snippets]

    # Iterate over each snippet and detect XSS vulnerabilities
    for snippet in all_snippets:
        if re.search(html_tag_pattern, snippet):
            html_tag_count += 1
        if re.search(concatenated_string_pattern, snippet):
            concatenated_string_count += 1

    if html_tag_count > 0:
        html_tag_count = 1
    if concatenated_string_count > 0:
        concatenated_string_count = 1

    return [html_tag_count, concatenated_string_count]  # Adjust to fit the expected output format

In [98]:
def command_injection(data):
    system_call_pattern = re.compile(r'\b(?:exec|system|shell_exec|passthru)\b', re.IGNORECASE)
    invoked_function_pattern = re.compile(r'\b(?:eval|create_function)\b', re.IGNORECASE)
    concatenated_string_pattern = re.compile(r'["\']\s*\.\s*["\']')

    system_call_count = 0
    invoked_function_count = 0
    concatenated_string_count = 0

    # Combine snippets from all categories into a single list
    all_snippets = [snippet for _, snippets in data for snippet in snippets]

    # Iterate over each snippet and detect command injection vulnerabilities
    for snippet in all_snippets:
        if re.search(system_call_pattern, snippet):
            system_call_count += 1
        if re.search(invoked_function_pattern, snippet):
            invoked_function_count += 1
        if re.search(concatenated_string_pattern, snippet):
            concatenated_string_count += 1

    if system_call_count > 0:
        system_call_count = 1
    if invoked_function_count > 0:
        invoked_function_count = 1
    if concatenated_string_count > 0:
        concatenated_string_count = 1

    return [system_call_count, invoked_function_count, concatenated_string_count]

In [99]:
def prototype_pollution(data):
    prototype_assignment_pattern = re.compile(r'Object\.prototype\.[\w$]+\s*=\s*.+')
    object_assignment_pattern = re.compile(r'([\w$]+|Object)\s*=\s*{[\w$]+:\s*.+,')
    object_manipulation_pattern = re.compile(r'Object\.(assign|setPrototypeOf)\s*\([\w$]+\s*,\s*{[\w$]+:\s*.+}\s*\)')
    json_parse_pattern = re.compile(r'JSON\.parse\s*\([\w$]+\s*\)')
    property_check_pattern = re.compile(r'\bif\s*\(\s*!\s*[\w$]+\s*\.hasOwnProperty\s*\(\s*[\w$]+\s*\)\s*\)\s*{')
    default_object_assignment_pattern = re.compile(r'[\w$]+\s*=\s*[\w$]+\s*\|\|\s*{};')
    dynamic_property_assignment_pattern = re.compile(r'[\w$]+\s*\[\s*[\w$]+\s*\]\s*=\s*.+')
    array_copy_pattern = re.compile(r'const\s+[\w$]+\s*=\s*[\w$]+\s*\[\s*[\w$]+\s*\];')

    prototype_assignment_count = 0
    object_assignment_count = 0
    object_manipulation_count = 0
    json_parse_count = 0
    property_check_count = 0
    default_object_assignment_count = 0
    dynamic_property_assignment_count = 0
    array_copy_count = 0

    all_snippets = [snippet for _, snippets in data for snippet in snippets]

    for snippet in all_snippets:
        if re.search(prototype_assignment_pattern, snippet):
            prototype_assignment_count += 1
        if re.search(object_assignment_pattern, snippet):
            object_assignment_count += 1
        if re.search(object_manipulation_pattern, snippet):
            object_manipulation_count += 1
        if re.search(json_parse_pattern, snippet):
            json_parse_count += 1
        if re.search(property_check_pattern, snippet):
            property_check_count += 1
        if re.search(default_object_assignment_pattern, snippet):
            default_object_assignment_count += 1
        if re.search(dynamic_property_assignment_pattern, snippet):
            dynamic_property_assignment_count += 1
        if re.search(array_copy_pattern, snippet):
            array_copy_count += 1

    return [prototype_assignment_count, object_assignment_count, object_manipulation_count,
            json_parse_count, property_check_count, default_object_assignment_count,
            dynamic_property_assignment_count, array_copy_count]


In [100]:
def file_inclusion(data):
    php_dynamic_inclusion_pattern = re.compile(r'(include|require)(_once)?\s*\(.*?\$_(GET|POST|REQUEST|COOKIE)', re.IGNORECASE)
    php_unsanitized_inclusion_pattern = re.compile(r'(include|require)(_once)?\s*\(.*?\$_(GET|POST|REQUEST|COOKIE)', re.IGNORECASE)
    php_allow_url_include_pattern = re.compile(r'allow_url_include\s*=\s*(1|On)', re.IGNORECASE)
    php_dynamic_path_pattern = re.compile(r'(include|require)(_once)?\s*\(.*?".*?\$_(GET|POST|REQUEST|COOKIE).*?\.\w+', re.IGNORECASE)
    php_directory_traversal_pattern = re.compile(r'(include|require)(_once)?\s*\(.*?\.+\.\.\/', re.IGNORECASE)
    php_sensitive_file_access_pattern = re.compile(r'(include|require)(_once)?\s*\(.*?(config|passwords|settings)\.php', re.IGNORECASE)

    js_dynamic_inclusion_pattern = re.compile(r'(import|require)\s*\(\s*.*?["\']\s*\+\s*(?:\$GET|\$POST|\$REQUEST|\$COOKIE)', re.IGNORECASE)
    js_unsanitized_inclusion_pattern = re.compile(r'(import|require)\s*\(\s*.*?["\']\s*\+\s*(?:\$GET|\$POST|\$REQUEST|\$COOKIE)', re.IGNORECASE)

    dynamic_inclusion_count = 0
    unsanitized_inclusion_count = 0
    php_allow_url_include_count = 0
    php_dynamic_path_count = 0
    php_directory_traversal_count = 0
    php_sensitive_file_access_count = 0

    all_snippets = [snippet for _, snippets in data for snippet in snippets]

    for snippet in all_snippets:
        dynamic_inclusion_count += len(re.findall(php_dynamic_inclusion_pattern, snippet)) + len(re.findall(js_dynamic_inclusion_pattern, snippet))
        unsanitized_inclusion_count += len(re.findall(php_unsanitized_inclusion_pattern, snippet)) + len(re.findall(js_unsanitized_inclusion_pattern, snippet))
        php_allow_url_include_count += len(re.findall(php_allow_url_include_pattern, snippet))
        php_dynamic_path_count += len(re.findall(php_dynamic_path_pattern, snippet))
        php_directory_traversal_count += len(re.findall(php_directory_traversal_pattern, snippet))
        php_sensitive_file_access_count += len(re.findall(php_sensitive_file_access_pattern, snippet))

    return [dynamic_inclusion_count, unsanitized_inclusion_count, php_allow_url_include_count,
            php_dynamic_path_count, php_directory_traversal_count, php_sensitive_file_access_count]


In [101]:
def authentication_bypass(data):
    weak_authentication_pattern = re.compile(r'(login|authenticate)\s*\(.*?\btrue\b', re.IGNORECASE)
    hardcoded_credentials_pattern = re.compile(r'(login|authenticate)\s*\(.*?[\'"].*?[\'"]\s*,\s*[\'"].*?[\'"]\s*\)', re.IGNORECASE)
    bypass_logic_pattern = re.compile(r'(if|else\s*if)\s*\(.*?authenticated\s*[=!]=\s*true', re.IGNORECASE)
    commented_out_authentication_pattern = re.compile(r'/\*.*?(login|authenticate).*?\btrue\b.*?\*/', re.IGNORECASE)

    # Initialize counters for authentication bypass patterns
    weak_authentication_count = 0
    hardcoded_credentials_count = 0
    bypass_logic_count = 0
    commented_out_authentication_count = 0

    # Combine snippets from all categories into a single list
    all_snippets = [snippet for _, snippets in data for snippet in snippets]

    # Iterate over each snippet and detect authentication bypass vulnerabilities
    for snippet in all_snippets:
        if re.search(weak_authentication_pattern, snippet):
            weak_authentication_count += 1
        if re.search(hardcoded_credentials_pattern, snippet):
            hardcoded_credentials_count += 1
        if re.search(bypass_logic_pattern, snippet):
            bypass_logic_count += 1
        if re.search(commented_out_authentication_pattern, snippet):
            commented_out_authentication_count += 1

    # Convert counts to 1 if vulnerabilities are detected, else keep them as 0
    weak_authentication_count = 1 if weak_authentication_count > 0 else 0
    hardcoded_credentials_count = 1 if hardcoded_credentials_count > 0 else 0
    bypass_logic_count = 1 if bypass_logic_count > 0 else 0
    commented_out_authentication_count = 1 if commented_out_authentication_count > 0 else 0

    # Return a single list containing the counts of patterns found
    return [weak_authentication_count, hardcoded_credentials_count, bypass_logic_count, commented_out_authentication_count]


In [102]:
def excessive_data(data):
    verbose_error_messages_pattern = re.compile(r'(error|exception|warning):\s*(.)', re.IGNORECASE)
    debugging_info_pattern = re.compile(r'(debug|trace):\s(.)', re.IGNORECASE)
    excessive_logging_pattern = re.compile(r'(log|write_to_file):\s(.)', re.IGNORECASE)
    excessive_response_data_pattern = re.compile(r'(response|output):\s(.)', re.IGNORECASE)
    insecure_data_storage_pattern = re.compile(r'(store|save)_data(.∗)(.∗)', re.IGNORECASE)
    insecure_transmission_pattern = re.compile(r'(http|ftp)://', re.IGNORECASE)
    client_side_data_exposure_pattern = re.compile(r'(api_key|auth_token):\s(.)', re.IGNORECASE)
    pii_disclosure_pattern = re.compile(r'(name|address|social_security|medical_record):\s(.)', re.IGNORECASE)
    system_config_disclosure_pattern = re.compile(r'(system_config|software_version|infrastructure):\s(.)', re.IGNORECASE)
    misconfigured_access_controls_pattern = re.compile(r'(access_control|authorization):\s(.*)', re.IGNORECASE)

    # Initialize counters for excessive data patterns
    verbose_error_messages_count = 0
    debugging_info_count = 0
    excessive_logging_count = 0
    excessive_response_data_count = 0
    insecure_data_storage_count = 0
    insecure_transmission_count = 0
    client_side_data_exposure_count = 0
    pii_disclosure_count = 0
    system_config_disclosure_count = 0
    misconfigured_access_controls_count = 0

    # Combine snippets from all categories into a single list
    all_snippets = [snippet for _, snippets in data for snippet in snippets]

    # Iterate over each snippet and detect excessive data vulnerabilities
    for snippet in all_snippets:
        if re.search(verbose_error_messages_pattern, snippet):
            verbose_error_messages_count += 1
        if re.search(debugging_info_pattern, snippet):
            debugging_info_count += 1
        if re.search(excessive_logging_pattern, snippet):
            excessive_logging_count += 1
        if re.search(excessive_response_data_pattern, snippet):
            excessive_response_data_count += 1
        if re.search(insecure_data_storage_pattern, snippet):
            insecure_data_storage_count += 1
        if re.search(insecure_transmission_pattern, snippet):
            insecure_transmission_count += 1
        if re.search(client_side_data_exposure_pattern, snippet):
            client_side_data_exposure_count += 1
        if re.search(pii_disclosure_pattern, snippet):
            pii_disclosure_count += 1
        if re.search(system_config_disclosure_pattern, snippet):
            system_config_disclosure_count += 1
        if re.search(misconfigured_access_controls_pattern, snippet):
            misconfigured_access_controls_count += 1

    # Convert counts to 1 if vulnerabilities are detected, else keep them as 0
    verbose_error_messages_count = 1 if verbose_error_messages_count > 0 else 0
    debugging_info_count = 1 if debugging_info_count > 0 else 0
    excessive_logging_count = 1 if excessive_logging_count > 0 else 0
    excessive_response_data_count = 1 if excessive_response_data_count > 0 else 0
    insecure_data_storage_count = 1 if insecure_data_storage_count > 0 else 0
    insecure_transmission_count = 1 if insecure_transmission_count > 0 else 0
    client_side_data_exposure_count = 1 if client_side_data_exposure_count > 0 else 0
    pii_disclosure_count = 1 if pii_disclosure_count > 0 else 0
    system_config_disclosure_count = 1 if system_config_disclosure_count > 0 else 0
    misconfigured_access_controls_count = 1 if misconfigured_access_controls_count > 0 else 0

    # Return a single list containing the counts of patterns found
    return [verbose_error_messages_count, debugging_info_count, excessive_logging_count,
            excessive_response_data_count, insecure_data_storage_count, insecure_transmission_count,
            client_side_data_exposure_count, pii_disclosure_count, system_config_disclosure_count,
            misconfigured_access_controls_count]

In [103]:
def compute_combined_matrix(matrix):
    combined_matrix = [1 if any(row) else 0 for row in matrix]
    return combined_matrix

In [104]:
all_snippets = [(var, snippets) for var, snippets in tainted_js_var_and_code] + [(var, snippets) for var, snippets in tainted_php_var_and_code]

# Detect vulnerabilities for all snippets
sqli_flags = sqli_vulnerability(all_snippets)
xss_flags = xss_vulnerability(all_snippets)
cmd_injection_flags = command_injection(all_snippets)
prototype_pollution_flags = prototype_pollution(all_snippets)
file_inclusion_flags = file_inclusion(all_snippets)
authentication_bypass_flags = authentication_bypass(all_snippets)
excessive_data_flags = excessive_data(all_snippets)

matrix = [
            list(sqli_flags),
            list(xss_flags),
            list(cmd_injection_flags),
            list(prototype_pollution_flags),
            list(file_inclusion_flags),
            list(authentication_bypass_flags),
            list(excessive_data_flags)
        ]

max_length = max(len(row) for row in matrix)
for row in matrix:
    while len(row) < max_length:
        row.append(0)

print("\nX-Feature:")
for row in matrix:
    print(row)

print("\nY-Feature:")
print(compute_combined_matrix(matrix))

NameError: name 'tainted_js_var_and_code' is not defined